In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [2]:
# directories
base_dirs = ["archive (1)\Training", "archive (1)\Testing"]

# subfolders
categories = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

# labels
labels = {'glioma_tumor': 0, 'meningioma_tumor': 0, 'no_tumor': 1, 'pituitary_tumor': 0}

# Lists to store the images and labels
images = []
image_labels = []

# loop
for base_dir in base_dirs:
    for category in categories:
        category_path = os.path.join(base_dir, category)
        for img_name in os.listdir(category_path): 
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')): # filter out non-image files
                img_path = os.path.join(category_path, img_name)
                img = Image.open(img_path)
                img = img.resize((128, 128))  # resize images for uniformity
                img = np.array(img) # convert to array
                images.append(img)
                image_labels.append(labels[category])

# convert lists to numpy arrays
images = np.array(images)
image_labels = np.array(image_labels)


In [3]:
# model
model = models.Sequential([
    # convolution layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    # dense layers
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid') 
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# split to test vs train
X_train, X_test, y_train, y_test = train_test_split(images, image_labels, test_size=0.2, random_state=42)

# Normalize the images
X_train = X_train / 255.0

X_test = X_test / 255.0

# Fit the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 50176)             0

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")